In [2]:
from __future__ import print_function #__future__ module is a built-in module in Python that is used to inherit new features that will be available in the new Python versions, from that it is importing module called print_function
import torch #PyTorch is a Python package that provides two high-level features: 1- Tensor computation (like NumPy) with strong GPU acceleration 2- Deep neural networks built on a tape-based autograd system
import torch.nn as nn #PyTorch provides the elegantly designed modules and classes torch.nn , torch.optim , Dataset , and DataLoader to help you create and train neural networks, 
                      #here (as nn) basically it is convention or short form use for torch.nn
import torch.nn.functional as F #in torch.nn there is a module called functional so we are calling that as F(which basically short form so we dont have to write complete 2 words again and again)  
import torch.optim as optim #torch.optim basically this package has various optimization algorithms example optim.SGD, optim.Adam
from torchvision import datasets, transforms #torchvision datasets for various videos and images, it is opensource

In [104]:
#so here our NN is created
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input 30x30 OUtput 28x28 RF 3x3  
                                                    #nn.Conv2d(1, 32, 3, padding=1) here I/P image is using 3x3 matrix/kernel and has 32 channels hence 32 kernels so like this maybe
                                                    # 30x30x1 | (3x3)x32 | 28x28x32 

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input 30x30 OUtput 28x28 RF 5x5  ## 30x30x32 | (3x3)x64 | 28x28x64 
        self.pool1 = nn.MaxPool2d(2, 2) #input 28x28 OUtput 14x14 RF 10x10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)#input 16x16 OUtput 14x14 RF 12x12 ##16x16x64 | (3x3)x128 | 14x14x128 
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)#input 16x16 OUtput 14x14 RF 14x14 ##16x16x128 | (3x3)x256 | 14x14x256
        self.pool2 = nn.MaxPool2d(2, 2) #input 14x14 OUtput 7x7 RF 28x28 
        self.conv5 = nn.Conv2d(256, 512, 3) #input 7x7 OUtput 5x5 RF 30x30 ##7x7x256 | (3x3)x512 | 14x14x512
        self.conv6 = nn.Conv2d(512, 1024, 3) #input 5x5 OUtput 3x3 RF 32x32 ##5x5x512 | (3x3)x1024 | 3x3x1024
        self.conv7 = nn.Conv2d(1024, 80, 3) #input 3x3 OUtput 1x1 RF 34x34 ##3x3x1024 | (3x3)x10 ? why we are using only 10 kernels of 3x3 strange let me ask sir | 1x1x10 

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x))))) #Network is created with activation function relu
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))) #Network is created with activation function relu
        x = F.relu(self.conv6(F.relu(self.conv5(x)))) #same relu function is applied in conv 5,6 and 7
        x = F.relu(self.conv7(x))
        x = x.view(-1, 80)
        return F.log_softmax(x)#I think in this we have use log to fall values between 0 and 1 or may be log probabilities

In [105]:
!pip install torchsummary #to install any module in python we use pip
from torchsummary import summary #this module provide summary of a model that we made #summary(model, input_size=(1, 28, 28)) this one which provide brief summary about our model
use_cuda = torch.cuda.is_available()#Returns a bool(means true or false) indicating if CUDA is currently available or not
device = torch.device("cuda" if use_cuda else "cpu") #if CUDA is not available use CPU
model = Net().to(device)#object will internally move all parameters and buffers to the device, dtype, or memory format so you wouldn’t need to reassign a model:
summary(model, input_size=(1, 28, 28))#provide summary of model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 80, 1, 1]         737,360
Total params: 7,024,976
Trainable params: 7,024,976
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 

<ipython-input-104-6762c46d0586>:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)#I think in this we have use log to fall values between 0 and 1 or may be log probabilities


In [106]:


torch.manual_seed(1)#basically used for generating random numbers
batch_size = 128 #image will be divided in batch size or maybe image will be processed in batch

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [107]:
from tqdm import tqdm #provides progress bar while training
def train(model, device, train_loader, optimizer, epoch):
    model.train()#now we will train our model using train function/module
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [108]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)#how we are deciding learning rate here as 0.01 #what is momentum here?

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)#train function which has many parameters like model, device, train loader etc
    test(model, device, test_loader)#now testing our model on unseen data basically

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-104-6762c46d0586>:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)#I think in this we have use log to fall values between 0 and 1 or may be log probabilities
loss=0.17707271873950958 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.79it/s]



Test set: Average loss: 0.0720, Accuracy: 9779/10000 (98%)



In [69]:
#self.conv7 = nn.Conv2d(1024, 80, 3)
#x = x.view(-1, 80)
#Test set: Average loss: 0.0680, Accuracy: 9796/10000 (98%)

In [70]:
#self.conv7 = nn.Conv2d(1024, 80, 3)
#x = x.view(-1, 80)
#Test set: Average loss: 0.0680, Accuracy: 9796/10000 (98%)
#Test set: Average loss: 0.0539, Accuracy: 9825/10000 (98%)
#strange accuracy is same but average loss is reduced

In [71]:
#self.conv7 = nn.Conv2d(1024, 240, 3)
#x = x.view(-1, 240)
#Test set: Average loss: 0.0512, Accuracy: 9831/10000 (98%)
#accuracy is same but loss is decreasing

In [72]:
#lol is increased batch size and accuracy decreases
#  0%|          | 0/235 [00:00<?, ?it/s]<ipython-input-61-c51a7e299078>:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
 # return F.log_softmax(x)#I think in this we have use log to fall values between 0 and 1 or may be log probabilities
#loss=0.11128437519073486 batch_id=234: 100%|██████████| 235/235 [00:16<00:00, 14.35it/s]

#Test set: Average loss: 0.0897, Accuracy: 9716/10000 (97%)

In [76]:
#   0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-61-c51a7e299078>:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
#   return F.log_softmax(x)#I think in this we have use log to fall values between 0 and 1 or may be log probabilities
# loss=0.028453771024942398 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 40.80it/s]

# Test set: Average loss: 0.1161, Accuracy: 9637/10000 (96%)


#again starange a reduced batch size to 64 and accuracy decreased

In [95]:
#even after this self.conv7 = nn.Conv2d(1024, 40, 3) accuracy not incresed only slight changes in loss

In [96]:
#even after this self.conv7 = nn.Conv2d(1024, 60, 3) accuracy 97%

In [110]:
#even after this self.conv7 = nn.Conv2d(1024, 60, 3) accuracy 97%


In [109]:
#even after this self.conv7 = nn.Conv2d(1024, 80, 3) accuracy 98% 
#after 80 accuracy is not increasing above 98 percent only loss in decreasing

In [112]:
#maybe real enginnering a nural network lies to increase accuracy above 98% hahahahahahhahahahahah